# Code to reproduce original_common_filtered.txt file from original catalogs

In [1]:
import obspy
import glob
import matplotlib.pyplot as plt
import numpy as np
import datetime
import matplotlib
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams['font.family'] = ['Arial']

## Read in all 3 catalogs

In [ ]:
og_cat = obspy.core.event.read_events('result_catalogs/endquakes_2017.xml')

In [ ]:
ml_cat = obspy.core.event.read_events('result_catalogs/ml_2017.xml',format='QUAKEML')

In [ ]:
tm_cat = obspy.core.event.read_events('result_catalogs/tm_2017.xml',format='QUAKEML')

## Filter catalogs

In [ ]:
# Filter template matching catalog
# Only keep those with cc sums of > 3.2 or < -3.2
keep_cat = obspy.core.event.catalog.Catalog()
for ev in tm_cat:
    s = ev.comments[2].text.split('=')[1]
    if float(s) > 3.2 or float(s) < -3.2:
        keep_cat.extend([ev])
tm_cat = keep_cat

In [ ]:
# Filter ML catalog
# Only keep those with picks on at least 3 stations
keep_cat = obspy.core.event.catalog.Catalog()
for ev in ml_cat:
    sta = [p.waveform_id.station_code for p in ev.picks]
    if len(np.unique(sta)) >= 3:
        keep_cat.extend([ev])
ml_cat = keep_cat

## Codes to compare them to each other and find common picks

In [2]:
def isolate_pick_times(cat,station,channel,phase):
    """
    cat is Obspy catalog
    station is station code, string
    channel is channel code, string
    
    returns a list of the pick times on that station and channel in the catalog
    and also the obspy catalog for the events those picks are from
    """
    pick_list = []
    ev_cat = obspy.core.event.catalog.Catalog()
    for ev in cat:
        if ev.picks[0].phase_hint != None:
            picktime = [pick.time.datetime for pick in ev.picks if (pick.waveform_id.station_code==station) & (pick.phase_hint==phase)]
        else:
            picktime = [pick.time.datetime for pick in ev.picks if (pick.waveform_id.station_code==station) & (pick.waveform_id.channel_code==channel)]
        
        
        if len(picktime) > 0:
            pick_list.append(picktime[0])
            ev_cat.append(ev)
            
    return(pick_list,ev_cat)

class PickComparison:
    def __init__(self):
        self.station = ''
        self.phase = ''
        self.channel = ''
        self.ml = []
        self.tm = []
        self.og = []
        self.tm_correct_cat = []
        self.ml_correct_cat = []
        
def compare_pick_times(og_cat,tm_cat,ml_cat):
    """
    """
    stations = ['KEMF','KEMF','KEMO','KEMO','NCHR','NCHR','ENWF','ENWF']
    channels = ['EHZ','EHE','EHZ','EHE','EHZ','EHE','HHZ','HHE']
    phases = ['P','S','P','S','P','S','P','S','P','S']
    
    comparisons = []
    
    for i in range(len(stations)):
        
        comparison = PickComparison()
        comparison.station = stations[i]
        comparison.phase = phases[i]
        comparison.channel = channels[i]
        
        print(stations[i])
        print(phases[i])
        og_times,ev_cat = isolate_pick_times(og_cat,stations[i],channels[i],phases[i])
        tm_times,scratch = isolate_pick_times(tm_cat,stations[i],channels[i],phases[i])
        ml_times,scratch = isolate_pick_times(ml_cat,stations[i],channels[i],phases[i])

        
        tm_correct = []
        tm_correct_cat = obspy.core.event.catalog.Catalog()
        for picktime in tm_times:
            timediff = [abs(og_picktime - picktime) for og_picktime in og_times]
            mintime = min(timediff)
            minind = np.argmin(timediff)
            if mintime < datetime.timedelta(seconds=0.5):
                tm_correct.append(picktime)
                tm_correct_cat.append(ev_cat[minind])
                
        ml_correct = []
        ml_correct_cat = obspy.core.event.catalog.Catalog()
        for picktime in ml_times:
            timediff = [abs(og_picktime - picktime) for og_picktime in og_times]
            mintime = min(timediff)
            minind = np.argmin(timediff)
            if mintime < datetime.timedelta(seconds=0.5):
                ml_correct.append(picktime)
                ml_correct_cat.append(ev_cat[minind])
        
                
        comparison.tm = tm_correct        
        comparison.ml = ml_correct
        comparison.tm_cat = tm_correct_cat
        comparison.ml_cat = ml_correct_cat
        comparison.og = og_times
        
        comparisons.append(comparison)
        
    return(comparisons)

## Calculate comparisons- this takes a while if catalogs are large

In [ ]:
comparisons = compare_pick_times(og_cat,tm_cat,ml_cat)

## Save to file

In [ ]:
import pickle
with open('original_common_filtered.txt', 'wb') as f:
    pickle.dump(comparisons,f) 